In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import (
    Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, 
    BatchNormalization, Dropout, Lambda, LSTM, Reshape
)
from tensorflow.keras import backend as K
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import tensorflow as tf

2025-10-29 14:21:38.491555: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-29 14:21:38.550788: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-29 14:21:40.315853: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [2]:
df = pd.read_csv('/home/dito-adistya/Dito/TA/Coding/LSTM-MBA/data/GSARPC3.csv')
df['smiles'] = df['smiles'].str.upper()

def encode_label(cat):
    if cat == "inhibitor":
        return 1
    elif cat == "neutral":
        return 0
    else:
        raise ValueError(f"Label tidak dikenal: {cat}")

df['target'] = df['categories'].apply(encode_label)

print("Distribusi label:")
print(df['target'].value_counts())

Distribusi label:
target
0    321
1    315
Name: count, dtype: int64


In [3]:
df_train, df_test = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df['target']
)
y_train = df_train['target'].values.astype(np.float32)
y_test = df_test['target'].values.astype(np.float32)

In [4]:
all_chars = set(''.join(df_train['smiles']))
charset = ['!', 'E', '?', '_'] + sorted(all_chars)
char_to_int = {c: i for i, c in enumerate(charset)}
vocab_size = len(charset)
max_len = max(len(s) for s in df_train['smiles']) + 2

def encode_smiles(smiles_list):
    sequences = []
    for s in smiles_list:
        seq = [char_to_int['!']]
        for c in s:
            seq.append(char_to_int.get(c, char_to_int['?']))
        seq.append(char_to_int['E'])
        if len(seq) < max_len:
            seq += [char_to_int['_']] * (max_len - len(seq))
        else:
            seq = seq[:max_len]
        sequences.append(seq)
    return np.array(sequences, dtype=np.int32)

X_train = encode_smiles(df_train['smiles'])
X_test = encode_smiles(df_test['smiles'])

unknown_count = np.sum(X_test == char_to_int['?'])
print(f"Unknown characters in test set: {unknown_count}")

truncated = sum(len(s) + 2 > max_len for s in df_test['smiles'])
print(f"Truncated test SMILES: {truncated}")

Unknown characters in test set: 0
Truncated test SMILES: 0


In [ ]:
inputs = Input(shape=(max_len,))
x = Embedding(vocab_size, 50, mask_zero=True)(inputs)
x = Conv1D(64, 10, activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = Conv1D(64, 5, activation='relu', padding='same')(x)
x = Conv1D(64, 3, activation='relu', padding='same')(x)
x = GlobalMaxPooling1D()(x)
x = Dropout(0.4)(x)
embedding = Dense(128, activation='relu')(x)

encoder = Model(inputs=inputs, outputs=embedding)
encoder.summary()

/home/dito-adistya/miniconda3/envs/py311/lib/python3.11/site-packages/keras/src/layers/layer.py:970: UserWarning: Layer 'conv1d_8' (of type Conv1D) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


Model: "functional_28"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)      │ (None, 165)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_3 (Embedding)         │ (None, 165, 32)        │           992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_8 (Conv1D)               │ (None, 165, 64)        │        20,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 165, 64)        │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_9 (Conv1D)               │ (None, 165, 64)        │        20,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_10 (Conv1D)              │ (None, 165, 64)        │        12,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_2          │ (None, 64)             │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │         8,320 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 63,008 (246.12 KB)

 Trainable params: 62,880 (245.62 KB)

 Non-trainable params: 128 (512.00 B)

In [16]:
train_emb = encoder.predict(X_train, verbose=1)
test_emb = encoder.predict(X_test, verbose=1)

16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


In [17]:
units = [128, 32]  
dropout_rate = 0.3
learning_rate = 0.0001
batch_size = 32
l2_reg = 1e-4

model_baseline_5 = Sequential()
model_baseline_5.add(tf.keras.Input(shape=(128,)))
model_baseline_5.add(tf.keras.layers.Reshape((1, 128)))
model_baseline_5.add(
    tf.keras.layers.LSTM(
        units[0],
        activation='tanh',
        recurrent_activation='sigmoid',
        return_sequences=True,
        kernel_regularizer=l2(l2_reg)
    )
)
model_baseline_5.add(
    tf.keras.layers.LSTM(
        units[1],
        activation='tanh',
        recurrent_activation='sigmoid',
        return_sequences=False,
        kernel_regularizer=l2(l2_reg)
    )
)
model_baseline_5.add(Dropout(dropout_rate))
model_baseline_5.add(Dense(1, activation='sigmoid'))

model_baseline_5.compile(
    optimizer=Adam(learning_rate=learning_rate),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC()]
)

history_baseline_5 = model_baseline_5.fit(
    train_emb, y_train,
    validation_data=(test_emb, y_test),
    epochs=100,
    batch_size=batch_size,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=1e-6
        )
    ],
    verbose=1
)

loss, acc, auc = model_baseline_5.evaluate(test_emb, y_test, verbose=0)
print(f"✅ Test Accuracy: {acc:.4f}, AUC: {auc:.4f}")

Epoch 1/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - accuracy: 0.4980 - auc_5: 0.5000 - loss: 0.7258 - val_accuracy: 0.5078 - val_auc_5: 0.5000 - val_loss: 0.7251 - learning_rate: 1.0000e-04
Epoch 2/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5039 - auc_5: 0.5000 - loss: 0.7246 - val_accuracy: 0.5078 - val_auc_5: 0.5000 - val_loss: 0.7239 - learning_rate: 1.0000e-04
Epoch 3/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5039 - auc_5: 0.5000 - loss: 0.7233 - val_accuracy: 0.5078 - val_auc_5: 0.5000 - val_loss: 0.7227 - learning_rate: 1.0000e-04
Epoch 4/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5039 - auc_5: 0.5000 - loss: 0.7221 - val_accuracy: 0.5078 - val_auc_5: 0.5000 - val_loss: 0.7215 - learning_rate: 1.0000e-04
Epoch 5/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5039 - auc_5: 0.5000 - loss: 0.7210 - val_accuracy: 0.5078 - val_auc_5: 0.5000 - val_loss: 0.7203 - learning_rate: 1.0000e-04
Epoch 6/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8m

In [ ]:
units = [128, 32]  
dropout_rate = 0.3
learning_rate = 0.0001
batch_size = 32
l2_reg = 1e-4

model_baseline_5 = Sequential()
model_baseline_5.add(tf.keras.Input(shape=(128,)))
model_baseline_5.add(tf.keras.layers.Reshape((1, 128)))
model_baseline_5.add(
    tf.keras.layers.LSTM(
        units[0],
        activation='tanh',
        recurrent_activation='sigmoid',
        return_sequences=True,
        kernel_regularizer=l2(l2_reg)
    )
)
model_baseline_5.add(
    tf.keras.layers.LSTM(
        units[1],
        activation='tanh',
        recurrent_activation='sigmoid',
        return_sequences=False,
        kernel_regularizer=l2(l2_reg)
    )
)
model_baseline_5.add(Dropout(dropout_rate))
model_baseline_5.add(Dense(1, activation='sigmoid'))

model_baseline_5.compile(
    optimizer=Adam(learning_rate=learning_rate),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC()]
)

history_baseline_5 = model_baseline_5.fit(
    train_emb, y_train,
    validation_data=(test_emb, y_test),
    epochs=100,
    batch_size=batch_size,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=5,
            min_lr=1e-6
        )
    ],
    verbose=1
)

loss, acc, auc = model_baseline_5.evaluate(test_emb, y_test, verbose=0)
print(f"✅ Test Accuracy: {acc:.4f}, AUC: {auc:.4f}")

In [5]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, LSTM, Dropout, Dense
# Impor lain yang dibutuhkan: BatchNormalization, Regularizers, dll.

def build_smiles2vec_unidirectional_lstm(
    max_len, 
    vocab_size, 
    embedding_dim, 
    lstm_units, 
    dropout_rate, 
    use_cnn=True,
    num_lstm_layers=2 # Bisa diubah/di-tune
):
    inputs = Input(shape=(max_len,))
    
    # Layer 1: Embedding (Input sekuensial)
    x = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputs) # Misalnya, 50 dim
    
    # Layer 2: CNN Pre-processing (Opsional, tapi ada di varian terbaik SMILES2vec)
    if use_cnn:
        # Lapisan Conv1D tunggal (atau bisa jadi dua lapisan Conv seperti di paper 
        # CNN-GRU/LSTM, tapi dengan GlobalMaxPooling dihapus)
        # Contoh: filter 192, kernel 3 (berdasarkan paper )
        x = Conv1D(filters=192, kernel_size=3, activation='relu', padding='same')(x)
        
    # Layer 3: JANTUNG TA - LSTM Unidirectional
    for i in range(num_lstm_layers):
        is_last_layer = (i == num_lstm_layers - 1)
        
        # return_sequences=True untuk semua layer kecuali layer terakhir
        x = LSTM(
            lstm_units, 
            activation='tanh', 
            recurrent_activation='sigmoid', 
            return_sequences=(not is_last_layer)
        )(x)
        
    # Layer 4: Dropout (Diterapkan setelah LSTM terakhir)
    x = Dropout(dropout_rate)(x)
    
    # Layer 5: Dense Output (Klasifikasi Biner)
    output = Dense(1, activation='sigmoid')(x) # Output aktivitas antiproliferatif (Inhibitor/Neutral)
    
    model = Model(inputs=inputs, outputs=output)
    return model

# --- PENGGUNAAN DALAM TA LO (Contoh Nilai yang Akan Di-tune MBA) ---
# Misalnya, kita asumsikan lo pakai 2 lapisan LSTM biasa (seperti yang diuji di paper [cite: 182])
MAX_LEN = 270 
VOCAB_SIZE = 50 
EMB_DIM = 50 
LSTM_UNITS_PER_LAYER = 256 # Ini adalah parameter utama MBA lo!
DROPOUT = 0.4
NUM_LAYERS = 2 # Ini juga bisa jadi parameter MBA lo

model_ta_unidir = build_smiles2vec_unidirectional_lstm(
    MAX_LEN, VOCAB_SIZE, EMB_DIM, LSTM_UNITS_PER_LAYER, DROPOUT, use_cnn=True, num_lstm_layers=NUM_LAYERS
)

model_ta_unidir.summary()

2025-10-29 11:21:59.265209: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/home/dito-adistya/miniconda3/envs/py311/lib/python3.11/site-packages/keras/src/layers/layer.py:970: UserWarning: Layer 'conv1d' (of type Conv1D) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 270)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 270, 50)        │         2,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 270, 192)       │        28,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 270, 256)       │       459,776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 256)            │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,016,837 (3.88 MB)

 Trainable params: 1,016,837 (3.88 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import numpy as np # Anggap data sudah ada dan siap

# --- HYPERPARAMETER AWAL (Akan di-tune oleh MBA lo) ---
MAX_LEN = 270 
VOCAB_SIZE = 50 
EMB_DIM = 50 
LSTM_UNITS_PER_LAYER = 256
DROPOUT = 0.4
NUM_LAYERS = 2
LEARNING_RATE = 1e-3 # Diambil dari setting default yang umum
BATCH_SIZE = 32 # Diambil dari setting di paper SMILES2vec
L2_REG = 1e-4

# --- Panggil Fungsi Pembuat Model ---
model_ta_unidir = build_smiles2vec_unidirectional_lstm(
    MAX_LEN, VOCAB_SIZE, EMB_DIM, LSTM_UNITS_PER_LAYER, DROPOUT, 
    use_cnn=True, num_lstm_layers=NUM_LAYERS
)

/home/dito-adistya/miniconda3/envs/py311/lib/python3.11/site-packages/keras/src/layers/layer.py:970: UserWarning: Layer 'conv1d_1' (of type Conv1D) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Embedding, Conv1D, LSTM, Dropout, Dense, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

# --- 1. DATA PREPARATION DAN ENCODING ---

# Load Data (Pastikan path file-nya benar)
# Catatan: Lo perlu ganti '/home/dito-adistya/Dito/TA/Coding/LSTM-MBA/data/GSARPC3.csv' dengan path yang benar di sistem lo.
df = pd.read_csv('/home/dito-adistya/Dito/TA/Coding/LSTM-MBA/data/GSARPC3.csv')
df['smiles'] = df['smiles'].str.upper()

# Fungsi Encoding Label (Target)
def encode_label(cat):
    if cat == "inhibitor":
        return 1
    elif cat == "neutral":
        return 0
    else:
        raise ValueError(f"Label tidak dikenal: {cat}")

df['target'] = df['categories'].apply(encode_label)

print("Distribusi label:")
print(df['target'].value_counts())

# Data Splitting (Train/Test)
df_train, df_test = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df['target']
)
y_train = df_train['target'].values.astype(np.float32)
y_test = df_test['target'].values.astype(np.float32)

# Vokabulari (Char-to-Int Mapping)
all_chars = set(''.join(df_train['smiles']))
# Token khusus: ! (Start), E (End), ? (Unknown), _ (Padding)
charset = ['!', 'E', '?', '_'] + sorted(all_chars) 
char_to_int = {c: i for i, c in enumerate(charset)}
vocab_size = len(charset)
max_len = max(len(s) for s in df_train['smiles']) + 2 # +2 untuk token Start dan End

print(f"\nMax SMILES Length: {max_len}")
print(f"Vocabulary Size: {vocab_size}")

# Fungsi Encoding SMILES (dengan Padding dan Start/End Token)
def encode_smiles(smiles_list, max_len, char_to_int):
    sequences = []
    # Ambil nilai int untuk token khusus dari dictionary
    int_start = char_to_int['!']
    int_end = char_to_int['E']
    int_unk = char_to_int['?']
    int_pad = char_to_int['_']
    
    for s in smiles_list:
        # Tambahkan Start token
        seq = [int_start]
        # Encode karakter
        for c in s:
            seq.append(char_to_int.get(c, int_unk))
        # Tambahkan End token
        seq.append(int_end)
        
        # Padding
        if len(seq) < max_len:
            seq += [int_pad] * (max_len - len(seq))
        # Truncation (Seharusnya tidak terjadi jika max_len dihitung dari max train)
        else:
            seq = seq[:max_len]
            
        sequences.append(seq)
        
    return np.array(sequences, dtype=np.int32)

# Encode Data Training dan Testing
X_train = encode_smiles(df_train['smiles'], max_len, char_to_int)
X_test = encode_smiles(df_test['smiles'], max_len, char_to_int)

# Cek hasil encoding
unknown_count = np.sum(X_test == char_to_int['?'])
print(f"\nUnknown characters in test set: {unknown_count}")
truncated = sum(len(s) + 2 > max_len for s in df_test['smiles'])
print(f"Truncated test SMILES: {truncated}")


# --- 2. MODEL ARCHITECTURE (SMILES2vec Unidirectional LSTM) ---

# --- HYPERPARAMETER AWAL ---
# Ini adalah nilai-nilai yang akan lo ganti dan dioptimasi oleh MBA!
MAX_LEN = X_train.shape[1] # Ambil max_len yang sudah dihitung
VOCAB_SIZE = vocab_size
EMB_DIM = 50 
LSTM_UNITS_PER_LAYER = 256
DROPOUT = 0.4
NUM_LAYERS = 2 
LEARNING_RATE = 1e-3
BATCH_SIZE = 32
L2_REG = 1e-4

# Fungsi Pembuat Model
def build_smiles2vec_unidirectional_lstm(
    max_len, vocab_size, embedding_dim, lstm_units, dropout_rate, use_cnn=True, num_lstm_layers=2
):
    inputs = Input(shape=(max_len,))
    
    # Layer 1: Embedding (dengan mask_zero=True untuk mengabaikan padding)
    x = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputs) 
    
    # Layer 2: CNN Pre-processing (CNN-LSTM di paper SMILES2vec)
    if use_cnn:
        # [cite_start]Contoh: filter 192, kernel 3 (berdasarkan varian terbaik di paper SMILES2vec) [cite: 197, 208, 227]
        x = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(x)
        
    # Layer 3: JANTUNG TA - LSTM Unidirectional
    for i in range(num_lstm_layers):
        is_last_layer = (i == num_lstm_layers - 1)
        
        # return_sequences=True untuk semua layer kecuali layer terakhir
        x = LSTM(
            lstm_units, 
            activation='tanh', 
            recurrent_activation='sigmoid', 
            return_sequences=(not is_last_layer)
            # Opsional: tambahkan kernel_regularizer=l2(L2_REG) jika dioptimasi
        )(x)
        
    # Layer 4: Dropout (Diterapkan setelah LSTM terakhir)
    x = Dropout(dropout_rate)(x)
    
    # Layer 5: Dense Output (Klasifikasi Biner)
    output = Dense(1, activation='sigmoid')(x) 
    
    model = Model(inputs=inputs, outputs=output)
    return model

# Panggil Model
model_ta_unidir = build_smiles2vec_unidirectional_lstm(
    MAX_LEN, VOCAB_SIZE, EMB_DIM, LSTM_UNITS_PER_LAYER, DROPOUT, 
    use_cnn=True, num_lstm_layers=NUM_LAYERS
)

# --- 3. KOMPILASI DAN TRAINING ---

# Kompilasi
model_ta_unidir.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')] 
)

# [cite_start]Callbacks (Early Stopping dan ReduceLROnPlateau) [cite: 162, 163]
callbacks_list = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10, 
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-6
    )
]

print("\n--- RINGKASAN MODEL ---")
model_ta_unidir.summary()

print("\n--- START TRAINING (Maksimum 250 Epoch) ---")
# Proses Training (menggunakan X_test sebagai validation_data)
history = model_ta_unidir.fit(
     X_train, y_train,
     validation_data=(X_test, y_test),
     epochs=250, # Jumlah maksimum epoch [cite: 161]
     batch_size=BATCH_SIZE,
     callbacks=callbacks_list,
     verbose=1
 )

print("\n--- HASIL AKHIR (TEST SET) ---")
# Evaluasi di Test Set
loss, acc, auc = model_ta_unidir.evaluate(X_test, y_test, verbose=0)
print(f"✅ Loss: {loss:.4f}, Accuracy: {acc:.4f}, AUC: {auc:.4f}")

Distribusi label:
target
0    321
1    315
Name: count, dtype: int64

Max SMILES Length: 165
Vocabulary Size: 31

Unknown characters in test set: 0
Truncated test SMILES: 0

--- RINGKASAN MODEL ---


/home/dito-adistya/miniconda3/envs/py311/lib/python3.11/site-packages/keras/src/layers/layer.py:970: UserWarning: Layer 'conv1d_2' (of type Conv1D) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 165)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_2 (Embedding)         │ (None, 165, 50)        │         1,550 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 165, 192)       │        28,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 165, 256)       │       459,776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 256)            │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,015,887 (3.88 MB)

 Trainable params: 1,015,887 (3.88 MB)

 Non-trainable params: 0 (0.00 B)


--- START TRAINING (Maksimum 250 Epoch) ---
Epoch 1/250
16/16 ━━━━━━━━━━━━━━━━━━━━ 13s 639ms/step - accuracy: 0.5059 - auc: 0.4819 - loss: 0.6947 - val_accuracy: 0.5078 - val_auc: 0.5000 - val_loss: 0.6931 - learning_rate: 0.0010
Epoch 2/250
16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 613ms/step - accuracy: 0.5157 - auc: 0.5124 - loss: 0.6940 - val_accuracy: 0.4922 - val_auc: 0.5162 - val_loss: 0.6969 - learning_rate: 0.0010
Epoch 3/250
16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 633ms/step - accuracy: 0.5157 - auc: 0.4845 - loss: 0.6951 - val_accuracy: 0.5078 - val_auc: 0.5159 - val_loss: 0.6932 - learning_rate: 0.0010
Epoch 4/250
16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 650ms/step - accuracy: 0.4488 - auc: 0.4538 - loss: 0.6945 - val_accuracy: 0.5234 - val_auc: 0.5322 - val_loss: 0.6928 - learning_rate: 0.0010
Epoch 5/250
16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 644ms/step - accuracy: 0.4980 - auc: 0.4887 - loss: 0.6931 - val_accuracy: 0.5234 - val_auc: 0.5109 - val_loss: 0.6926 - learning_rate: 0.0010
Epoch 6/250
16/16 ━━━━━━━

KeyboardInterrupt: 

In [15]:
from tensorflow.keras.layers import BatchNormalization

def build_smiles2vec_lstm_light(
    max_len, vocab_size, embedding_dim, lstm_units, dropout_rate, 
    cnn_filters=64, num_lstm_layers=1 # Default 1 layer
):
    inputs = Input(shape=(max_len,))
    
    # Layer 1: Embedding (Ringan)
    x = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputs) 
    
    # Layer 2: CNN Pre-processing (Ringan + BN)
    # Kita hanya pakai 1 lapisan CNN
    x = Conv1D(filters=cnn_filters, kernel_size=3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x) # Penting untuk data kecil/batch kecil
        
    # Layer 3: JANTUNG TA - LSTM Unidirectional (Ringan)
    for i in range(num_lstm_layers):
        is_last_layer = (i == num_lstm_layers - 1)
        
        x = LSTM(
            lstm_units, 
            activation='tanh', 
            recurrent_activation='sigmoid', 
            return_sequences=(not is_last_layer)
        )(x)
        
    # Layer 4: Dropout
    x = Dropout(dropout_rate)(x)
    
    # Layer 5: Dense Output
    output = Dense(1, activation='sigmoid')(x) 
    
    model = Model(inputs=inputs, outputs=output)
    return model

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    InputLayer, Conv1D, LSTM, Dense, Dropout, Activation
)
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K

# --- 1. Load dan Preprocess Data ---
df = pd.read_csv('/home/dito-adistya/Dito/TA/Coding/LSTM-MBA/data/GSARPC3.csv')
df['smiles'] = df['smiles'].str.upper()

def encode_label(cat):
    return 1 if cat == "inhibitor" else 0

df['target'] = df['categories'].apply(encode_label)
print("Distribusi label:")
print(df['target'].value_counts())

# Split data (stratified)
df_train, df_test = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df['target']
)

# --- 2. Bangun vocabulary dan encode SMILES ---
all_chars = set(''.join(df_train['smiles']))
charset = sorted(all_chars)
# Tambahkan token khusus jika perlu, tapi SMILES2vec asli TIDAK pakai embedding/token khusus
# Jadi kita langsung one-hot encode karakter asli
char_to_int = {c: i for i, c in enumerate(charset)}
vocab_size = len(charset)
max_len = max(len(s) for s in df['smiles'])  # gunakan seluruh data untuk max_len
# Sesuai makalah: max_len = 250, padding ±10 di kiri/kanan → kita sederhanakan jadi padding kanan saja
MAX_LEN = min(max_len, 250)  # batasi maks 250

def smiles_to_onehot(smiles_list, char_to_int, max_len):
    """
    Konversi list SMILES ke one-hot tensor: (n, max_len, vocab_size)
    """
    n = len(smiles_list)
    X = np.zeros((n, max_len, len(char_to_int)), dtype=np.float32)
    for i, s in enumerate(smiles_list):
        s = s[:max_len]  # truncate jika terlalu panjang
        for t, char in enumerate(s):
            if char in char_to_int:
                X[i, t, char_to_int[char]] = 1.0
            # else: biarkan 0 (tidak ada karakter unknown karena pakai train set untuk vocab)
    return X

X_train = smiles_to_onehot(df_train['smiles'], char_to_int, MAX_LEN)
X_test = smiles_to_onehot(df_test['smiles'], char_to_int, MAX_LEN)
y_train = df_train['target'].values.astype(np.float32)
y_test = df_test['target'].values.astype(np.float32)

print(f"Input shape: {X_train.shape}")
print(f"Vocab size: {vocab_size}, Max length: {MAX_LEN}")

# --- 3. Fungsi Build Model (untuk MBO) ---
def build_model(
    use_conv=True,
    conv_filters=192,
    conv_kernel_size=3,
    lstm_units_1=128,
    lstm_units_2=64,
    lstm_units_3=32,
    dropout_rate=0.4,
    learning_rate=0.001
):
    """
    Bangun model Conv1D → LSTM (atau LSTM saja) untuk prediksi biner.
    Input: one-hot encoded SMILES (shape: [None, MAX_LEN, vocab_size])
    """
    model = Sequential()
    model.add(InputLayer(input_shape=(MAX_LEN, vocab_size)))

    # Opsional: Conv1D layer
    if use_conv:
        model.add(Conv1D(
            filters=conv_filters,
            kernel_size=conv_kernel_size,
            activation='relu',
            padding='same'
        ))

    # LSTM layers
    model.add(LSTM(lstm_units_1, return_sequences=True))
    model.add(LSTM(lstm_units_2, return_sequences=False))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))  # binary classification

    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

# --- 4. Contoh Penggunaan Model ---
if __name__ == "__main__":
    # Contoh: build model dengan Conv1D
    model = build_model(
        use_conv=True,
        conv_filters=192,
        conv_kernel_size=3,
        lstm_units_1=64,
        lstm_units_2=32,
        dropout_rate=0.3,
        learning_rate=0.001
    )
    model.summary()

    # Latih model (hanya contoh, nanti diganti oleh MBO loop)
    history = model.fit(
        X_train, y_train,
        batch_size=32,
        epochs=100,
        validation_data=(X_test, y_test),
        verbose=1
    )

    # Evaluasi
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
    print(f"\nTest Accuracy: {test_acc:.4f}")

Distribusi label:
target
0    321
1    315
Name: count, dtype: int64
Input shape: (508, 163, 27)
Vocab size: 27, Max length: 163


/home/dito-adistya/miniconda3/envs/py311/lib/python3.11/site-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_2 (Conv1D)               │ (None, 163, 192)       │        15,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 163, 64)        │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 93,985 (367.13 KB)

 Trainable params: 93,985 (367.13 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 131ms/step - accuracy: 0.4902 - loss: 0.6983 - val_accuracy: 0.4922 - val_loss: 0.6928
Epoch 2/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 111ms/step - accuracy: 0.4980 - loss: 0.6929 - val_accuracy: 0.5078 - val_loss: 0.6930
Epoch 3/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 102ms/step - accuracy: 0.5079 - loss: 0.6928 - val_accuracy: 0.5078 - val_loss: 0.6930
Epoch 4/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 103ms/step - accuracy: 0.5236 - loss: 0.6933 - val_accuracy: 0.5156 - val_loss: 0.6932
Epoch 5/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 107ms/step - accuracy: 0.5118 - loss: 0.6925 - val_accuracy: 0.5156 - val_loss: 0.6933
Epoch 6/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 110ms/step - accuracy: 0.4469 - loss: 0.6945 - val_accuracy: 0.5156 - val_loss: 0.6935
Epoch 7/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 121ms/step - accuracy: 0.5118 - loss: 0.6925 - val_accuracy: 0.5156 - val_loss: 0.6930
Epoch 8/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 108ms/step - accuracy: 0.5157 - loss: 0.6916 - val_accu

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    InputLayer, Conv1D, LSTM, Dense, Dropout, Activation
)
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K

# --- 1. Load dan Preprocess Data ---
df = pd.read_csv('/home/dito-adistya/Dito/TA/Coding/LSTM-MBA/data/GSARPC3.csv')
df['smiles'] = df['smiles'].str.upper()

def encode_label(cat):
    return 1 if cat == "inhibitor" else 0

df['target'] = df['categories'].apply(encode_label)
print("Distribusi label:")
print(df['target'].value_counts())

# Split data (stratified)
df_train, df_test = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df['target']
)

# --- 2. Bangun vocabulary dan encode SMILES ---
all_chars = set(''.join(df_train['smiles']))
charset = sorted(all_chars)
# Tambahkan token khusus jika perlu, tapi SMILES2vec asli TIDAK pakai embedding/token khusus
# Jadi kita langsung one-hot encode karakter asli
char_to_int = {c: i for i, c in enumerate(charset)}
vocab_size = len(charset)
max_len = max(len(s) for s in df['smiles'])  # gunakan seluruh data untuk max_len
# Sesuai makalah: max_len = 250, padding ±10 di kiri/kanan → kita sederhanakan jadi padding kanan saja
MAX_LEN = min(max_len, 250)  # batasi maks 250

def smiles_to_onehot(smiles_list, char_to_int, max_len):
    """
    Konversi list SMILES ke one-hot tensor: (n, max_len, vocab_size)
    """
    n = len(smiles_list)
    X = np.zeros((n, max_len, len(char_to_int)), dtype=np.float32)
    for i, s in enumerate(smiles_list):
        s = s[:max_len]  # truncate jika terlalu panjang
        for t, char in enumerate(s):
            if char in char_to_int:
                X[i, t, char_to_int[char]] = 1.0
            # else: biarkan 0 (tidak ada karakter unknown karena pakai train set untuk vocab)
    return X

X_train = smiles_to_onehot(df_train['smiles'], char_to_int, MAX_LEN)
X_test = smiles_to_onehot(df_test['smiles'], char_to_int, MAX_LEN)
y_train = df_train['target'].values.astype(np.float32)
y_test = df_test['target'].values.astype(np.float32)

print(f"Input shape: {X_train.shape}")
print(f"Vocab size: {vocab_size}, Max length: {MAX_LEN}")

# --- 3. Fungsi Build Model (untuk MBO) ---
def build_model(
    use_conv=True,
    conv_filters=192,
    conv_kernel_size=3,
    lstm_units_1=128,
    lstm_units_2=64,
    lstm_units_3=32,
    dropout_rate=0.4,
    learning_rate=0.001
):
    """
    Bangun model Conv1D → LSTM (atau LSTM saja) untuk prediksi biner.
    Input: one-hot encoded SMILES (shape: [None, MAX_LEN, vocab_size])
    """
    model = Sequential()
    model.add(InputLayer(input_shape=(MAX_LEN, vocab_size)))

    # Opsional: Conv1D layer
    if use_conv:
        model.add(Conv1D(
            filters=conv_filters,
            kernel_size=conv_kernel_size,
            activation='relu',
            padding='same'
        ))

    # LSTM layers
    model.add(LSTM(lstm_units_1, return_sequences=True))
    model.add(LSTM(lstm_units_2, return_sequences=False))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))  # binary classification

    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

# --- 4. Contoh Penggunaan Model ---
if __name__ == "__main__":
    # Contoh: build model dengan Conv1D
    model = build_model(
        use_conv=True,
        conv_filters=192,
        conv_kernel_size=3,
        lstm_units_1=32,
        lstm_units_2=16,
        dropout_rate=0.3,
        learning_rate=0.001
    )
    model.summary()

    # Latih model (hanya contoh, nanti diganti oleh MBO loop)
    history = model.fit(
        X_train, y_train,
        batch_size=32,
        epochs=100,
        validation_data=(X_test, y_test),
        verbose=1
    )

    # Evaluasi
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
    print(f"\nTest Accuracy: {test_acc:.4f}")

Distribusi label:
target
0    321
1    315
Name: count, dtype: int64
Input shape: (508, 163, 27)
Vocab size: 27, Max length: 163


/home/dito-adistya/miniconda3/envs/py311/lib/python3.11/site-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_1 (Conv1D)               │ (None, 163, 192)       │        15,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 163, 32)        │        28,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 16)             │         3,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 47,697 (186.32 KB)

 Trainable params: 47,697 (186.32 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 116ms/step - accuracy: 0.4961 - loss: 0.6935 - val_accuracy: 0.4922 - val_loss: 0.6935
Epoch 2/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - accuracy: 0.4882 - loss: 0.6935 - val_accuracy: 0.5078 - val_loss: 0.6937
Epoch 3/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 90ms/step - accuracy: 0.5098 - loss: 0.6930 - val_accuracy: 0.5156 - val_loss: 0.6935
Epoch 4/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 88ms/step - accuracy: 0.4705 - loss: 0.6938 - val_accuracy: 0.5156 - val_loss: 0.6935
Epoch 5/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 87ms/step - accuracy: 0.5059 - loss: 0.6930 - val_accuracy: 0.5156 - val_loss: 0.6934
Epoch 6/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 97ms/step - accuracy: 0.4606 - loss: 0.6932 - val_accuracy: 0.4922 - val_loss: 0.6945
Epoch 7/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 111ms/step - accuracy: 0.4980 - loss: 0.6929 - val_accuracy: 0.4922 - val_loss: 0.6943
Epoch 8/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 108ms/step - accuracy: 0.5059 - loss: 0.6925 - val_accuracy:

In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Embedding, Conv1D, LSTM, Dense, Dropout
)
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# --- 1. Load dan Preprocess Data ---
df = pd.read_csv('/home/dito-adistya/Dito/TA/Coding/LSTM-MBA/data/GSARPC3.csv')
df['smiles'] = df['smiles'].str.upper()

def encode_label(cat):
    return 1 if cat == "inhibitor" else 0

df['target'] = df['categories'].apply(encode_label)
print("Distribusi label:")
print(df['target'].value_counts())

# Split data (stratified)
df_train, df_test = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df['target']
)

# --- 2. Bangun vocabulary dan encode SMILES ke integer ---
all_chars = set(''.join(df_train['smiles']))
charset = sorted(all_chars)
char_to_int = {c: i for i, c in enumerate(charset)}
vocab_size = len(charset)
MAX_LEN = min(max(len(s) for s in df['smiles']), 250)  # batas maks 250

def smiles_to_int(smiles_list, char_to_int, max_len):
    """
    Konversi list SMILES ke integer sequence: (n, max_len)
    Karakter tidak dikenal diisi 0 (bisa diganti dengan token khusus jika mau)
    """
    n = len(smiles_list)
    X = np.zeros((n, max_len), dtype=np.int32)
    for i, s in enumerate(smiles_list):
        s = s[:max_len]  # truncate
        for t, char in enumerate(s):
            X[i, t] = char_to_int.get(char, 0)  # fallback ke 0 jika unknown
    return X

X_train = smiles_to_int(df_train['smiles'], char_to_int, MAX_LEN)
X_test = smiles_to_int(df_test['smiles'], char_to_int, MAX_LEN)
y_train = df_train['target'].values.astype(np.float32)
y_test = df_test['target'].values.astype(np.float32)

print(f"Input shape: {X_train.shape}")
print(f"Vocab size: {vocab_size}, Max length: {MAX_LEN}")

# --- 3. Fungsi Build Model dengan Embedding (untuk MBO) ---
def build_model(
    embedding_dim=50,
    use_conv=True,
    conv_filters=192,
    conv_kernel_size=3,
    lstm_units_1=128,
    lstm_units_2=64,
    dropout_rate=0.4,
    learning_rate=0.001
):
    """
    Bangun model: Embedding → [Conv1D] → LSTM → Dense
    Input: integer-encoded SMILES (shape: [None, MAX_LEN])
    """
    model = Sequential()
    model.add(Embedding(
        input_dim=vocab_size,
        output_dim=embedding_dim,
        input_length=MAX_LEN
    ))

    if use_conv:
        model.add(Conv1D(
            filters=conv_filters,
            kernel_size=conv_kernel_size,
            activation='relu',
            padding='same'
        ))

    model.add(LSTM(lstm_units_1, return_sequences=True))
    model.add(LSTM(lstm_units_2, return_sequences=False))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

# --- 4. Contoh Penggunaan Model ---
if __name__ == "__main__":
    model = build_model(
        embedding_dim=50,
        use_conv=True,
        conv_filters=64,        # lebih kecil untuk data kecil
        conv_kernel_size=3,
        lstm_units_1=64,
        lstm_units_2=32,
        dropout_rate=0.3,
        learning_rate=0.001
    )
    model.summary()

    history = model.fit(
        X_train, y_train,
        batch_size=32,
        epochs=100,
        validation_data=(X_test, y_test),
        verbose=1
    )

    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
    print(f"\nTest Accuracy: {test_acc:.4f}")

Distribusi label:
target
0    321
1    315
Name: count, dtype: int64
Input shape: (508, 163)
Vocab size: 27, Max length: 163


/home/dito-adistya/miniconda3/envs/py311/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_8 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_9 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 5s 138ms/step - accuracy: 0.4941 - loss: 0.6938 - val_accuracy: 0.5078 - val_loss: 0.6930
Epoch 2/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 109ms/step - accuracy: 0.5020 - loss: 0.6934 - val_accuracy: 0.5078 - val_loss: 0.6930
Epoch 3/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 117ms/step - accuracy: 0.5236 - loss: 0.6932 - val_accuracy: 0.5078 - val_loss: 0.6931
Epoch 4/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 107ms/step - accuracy: 0.4941 - loss: 0.6950 - val_accuracy: 0.5156 - val_loss: 0.6931
Epoch 5/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 120ms/step - accuracy: 0.4921 - loss: 0.6932 - val_accuracy: 0.5078 - val_loss: 0.6930
Epoch 6/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 113ms/step - accuracy: 0.4803 - loss: 0.6930 - val_accuracy: 0.5078 - val_loss: 0.6930
Epoch 7/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 117ms/step - accuracy: 0.4961 - loss: 0.6933 - val_accuracy: 0.5078 - val_loss: 0.6931
Epoch 8/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 112ms/step - accuracy: 0.4921 - loss: 0.6935 - val_accu